In [ ]:
import sire as sr
import glob
import numpy as np
import matplotlib.pyplot as plt

from multiprocessing import Pool

import logging
from rdkit import rdBase

In [ ]:
dihedral = "1_2_3_4"
seed_no = 4
fragment = 1

n_cpus = 26

In [ ]:
n_confs = 50 # number of conformers scanned
conf_numbers = np.arange(0,n_confs-1).tolist()

In [ ]:
idx1, idx2, idx3, idx4 = map(int, dihedral.split("_"))
print(idx1,idx2,idx3,idx4)

In [ ]:
forward = np.arange(-180,190,10).tolist()
energies_per_dihedral = []

Getting the energy of the molecule minus the energy of the torsion to be optimised

In [ ]:
# Set RDKit logger level to WARNING or ERROR
rdBase.LogToPythonStderr()
logger = logging.getLogger("rdkit")
logger.setLevel(logging.ERROR)  # or logging.WARNING

from IPython.display import display, HTML

display(HTML("""
<style>
.output_subarea.output_javascript_error {
    display: none !important;
}
</style>
"""))


def compute_mm_energies(dihe):
    print(f'{dihe}'+"\n")
    mm_energies = []
    
    file_patterns_prmtop = [
        'profiles_torsions/optimised_geometries/fragment%s/torsion%s/torsion%s_seed%s_conf%s_dihe_%s*_resp_charges_atomtypes_renamed.mol2.prmtop' % 
        (fragment, dihedral, dihedral, seed_no, conf, dihe) for conf in conf_numbers
    ]
    
    file_patterns_inpcrd = [
        'profiles_torsions/optimised_geometries/fragment%s/torsion%s/torsion%s_seed%s_conf%s_dihe_%s*_resp_charges_atomtypes_renamed.mol2.inpcrd' % 
        (fragment, dihedral, dihedral, seed_no, conf, dihe) for conf in conf_numbers
    ]

    file_list_prmtop = []
    file_list_inpcrd = []

    for pattern_prmtop, pattern_inpcrd in zip(file_patterns_prmtop, file_patterns_inpcrd):
        file_list_prmtop.extend(glob.glob(pattern_prmtop))
        file_list_inpcrd.extend(glob.glob(pattern_inpcrd))

    for fle in file_list_inpcrd:
        mol = sr.load(file_list_prmtop[0], fle)
        mm_energies.append(mol.energy().value())  # energies in kcal/mol

    return mm_energies

if __name__ == '__main__':
    with Pool(processes=n_cpus):  
        energies_per_dihedral = Pool(n_cpus).map(compute_mm_energies, forward)


In [ ]:
# optional: load one of the mol2 files from the QM scans to check the torsion potential expression for the torsion under study
# or any other torsion (e.g. 2-3-4-5)

mol = sr.load(
    "profiles_torsions/optimised_geometries/fragment%s/torsion%s/torsion%s_seed4_conf0_dihe_-90_2_forward_resp_charges_atomtypes_renamed.mol2.prmtop" % (fragment, dihedral, dihedral),
    "profiles_torsions/optimised_geometries/fragment%s/torsion%s/torsion%s_seed4_conf0_dihe_-90_2_forward_resp_charges_atomtypes_renamed.mol2.inpcrd" % (fragment, dihedral, dihedral)
)

In [ ]:
torsion = mol.dihedrals(f"atomnum {idx1}", f"atomnum {idx2}", f"atomnum {idx3}", f"atomnum {idx4}")
torsion.potentials()

In [ ]:
torsion = mol.dihedrals(f"atomnum {2}", f"atomnum {3}", f"atomnum {4}", f"atomnum {5}")
torsion.potentials()

In [ ]:
energies_per_dihedral

In [ ]:
avg_energy_per_dihedral = [] # saving average energy per dihedral for all conformers

for i in energies_per_dihedral: 
    avg_energy_per_dihedral.append(np.mean(i))

avg_energy_per_dihedral

In [ ]:
def ensure_python_list(data):
    if isinstance(data, list):
        return [float(x) if isinstance(x, (np.floating, np.integer)) else x for x in data]

    elif isinstance(data, np.ndarray):
        return data.astype(float).tolist()

    elif isinstance(data, (np.floating, np.integer)):
        return [float(data)]

    return data

In [ ]:
avg_energy_per_dihedral = ensure_python_list(avg_energy_per_dihedral)
avg_energy_per_dihedral

In [ ]:
# normalise average energy and check if values for -180 and 180 are missing (usually -180 is always missing)

norm_avg_energy_per_dihedral = []

def is_missing(x):
    return (isinstance(x, list) and len(x) == 0) or (isinstance(x, float) and np.isnan(x))

# Check first and last elements (values for -180 and 180 torsions likely to be missing)
if is_missing(avg_energy_per_dihedral[0]) and is_missing(avg_energy_per_dihedral[-1]):
    valid = avg_energy_per_dihedral[1:-1]
elif is_missing(avg_energy_per_dihedral[0]):
    valid = avg_energy_per_dihedral[1:]
else:
    valid = avg_energy_per_dihedral

valid_numeric = [x for x in valid if not is_missing(x)]
min_val = np.min(valid_numeric)

norm_avg_energy_per_dihedral = [x - min_val for x in valid_numeric]
norm_avg_energy_per_dihedral


In [ ]:
np.save('profiles_torsions/individual_conformer_scans/fragment%s/torsion%s/torsion%s_seed%s_mm.npy' % (fragment, dihedral, dihedral, seed_no), norm_avg_energy_per_dihedral)

In [ ]:
import matplotlib.pyplot as plt
forward = list(range(-170, 181, 10)) # here -180 has been excluded as no MM energy was available, modify according to your data
idx = 0
for i in norm_avg_energy_per_dihedral:
    plt.plot(forward[idx], i, marker='o', linestyle='None', color='blue')
    idx += 1

plt.xlabel('Torsion / $\circ$', fontsize=15)
plt.ylabel('E$_{MM,total}$ - E$_{MM,torsion}$ / kcal mol$^{-1}$', fontsize=15)

plt.xticks(range(-180, 181, 60), fontsize=12) 
plt.yticks(fontsize=12) 

plt.xlim(-185, 185)
plt.ylim(0.0,4.0)
plt.rcParams['figure.dpi'] = 300
plt.tight_layout()
plt.show()
